In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
os.chdir('/content/drive/MyDrive/Final Year Project/Data')
!pwd

/content/drive/MyDrive/Final Year Project/Data


# Metar weather data collection

 from https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=2025&mes=02&day=09&hora=14&anof=2025&mesf=02&dayf=10&horaf=14&minf=59&send=send using data scraping


In [ ]:
pip install metar

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL for Ogimet METAR data
url = "https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=2024&mes=02&day=01&hora=01&anof=2024&mesf=02&dayf=29&horaf=23&minf=59&send=send"
# Request page content
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Extract METAR strings from <pre> tags
metar_strings = [pre.get_text().strip() for pre in soup.find_all('pre')]

# Display extracted METAR data
for metar in metar_strings:
    print(metar)

METAR EIDW 292330Z 13007KT 100V160 9999 FEW022 SCT060 BKN120 03/M01 Q0992 NOSIG=
METAR EIDW 292300Z 15007KT 120V190 9999 FEW022 BKN120 04/M01 Q0993 NOSIG=
METAR EIDW 292230Z 15007KT 9999 FEW022 BKN140 04/M02 Q0993 NOSIG=
METAR EIDW 292200Z 13005KT 9999 FEW022 SCT140 BKN170 03/M02 Q0994 NOSIG=
METAR EIDW 292130Z 17007KT 9999 FEW022 02/M01 Q0995 NOSIG=
METAR EIDW 292100Z 18007KT 9999 FEW022 02/M01 Q0995 NOSIG=
METAR EIDW 292030Z 19007KT 9999 FEW022 02/M01 Q0996 NOSIG=
METAR EIDW 292000Z 20007KT 170V230 9999 FEW015 03/M00 Q0997 NOSIG=
METAR EIDW 291930Z 20007KT 170V230 9999 FEW015 03/M00 Q0997 NOSIG=
METAR EIDW 291900Z 19006KT 160V220 9999 FEW015 03/01 Q0997 NOSIG=
METAR EIDW 291830Z 22006KT 190V270 9999 FEW016 03/01 Q0998 NOSIG=
METAR EIDW 291800Z 23007KT 180V270 9999 FEW018CB 03/01 Q0998 NOSIG=
METAR EIDW 291730Z 23006KT 190V270 9999 -SHRA FEW018CB SCT048 04/02 Q0998 NOSIG=
METAR EIDW 291700Z 28010KT 9999 -SHRA FEW018CB SCT026 03/02 Q0999 NOSIG=
METAR EIDW 291630Z 24008G18KT 190V280 999

THIS CODE BELOW WORKS FOR INDIVIDUAL DATA SCRAPING FROM THE LIST OF LINKS THAT I HAVE STORED ON LAPTOP

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL for 2023 METAR data
url = "https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=ALL&ord=DIR&nil=SI&fmt=html&ano=2024&mes=12&day=01&hora=00&anof=2024&mesf=12&dayf=31&horaf=23&minf=59&send=send"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# Make the request with headers
response = requests.get(url, headers=headers)

# Parse the page content
soup = BeautifulSoup(response.text, 'html.parser')

# Extract METAR strings from <pre> tags
metar_strings = [td.get_text().strip() for td in soup.find_all('pre')]

# Verify scraped METARs
print(f"Total METARs found: {len(metar_strings)}")
print("First few METARs:", metar_strings[:5])




Total METARs found: 1640
First few METARs: ['METAR EIDW 010000Z 18012KT 9999 FEW006 SCT012 BKN018 14/11 Q1008 NOSIG=', 'METAR EIDW 010030Z 17009KT 9999 FEW006 SCT012 BKN018 14/11 Q1007 NOSIG=', 'METAR EIDW 010100Z 17008KT 140V200 9999 FEW006 SCT012 BKN018 13/11 Q1007 NOSIG=', 'METAR EIDW 010130Z 19008KT 150V240 9999 -DZ FEW006 SCT012 BKN018 13/11 Q1007 NOSIG=', 'METAR EIDW 010200Z 20008KT 9999 FEW006 SCT012 BKN018 13/11 Q1007 NOSIG=']


In [ ]:
import numpy as np

def calculate_humidity(temp, dew_point):
    if temp == "N/A" or dew_point == "N/A":
        return "N/A"
    return round(100 * (np.exp((17.625 * dew_point) / (243.04 + dew_point)) / np.exp((17.625 * temp) / (243.04 + temp))), 2)


In [ ]:
import pandas as pd
from metar import Metar  # Ensure you have `metar` installed via pip

data = []

for metar_string in metar_strings:
    try:
        metar = Metar.Metar(metar_string)  # Parse METAR data
        data.append({
            "Station": metar.station_id,
            "Observation Time": metar.time.strftime('%Y-%m-%d %H:%M:%S') if metar.time else "N/A",
            "Wind Speed (KT)": metar.wind_speed.value("KT") if metar.wind_speed else "N/A",
            "Visibility (m)": metar.vis.value("m") if metar.vis else "N/A",
            "Temperature (C)": metar.temp.value("C") if metar.temp else "N/A",
            "Dew Point (C)": metar.dewpt.value("C") if metar.dewpt else "N/A",
            "Pressure (hPa)": metar.press.value("hPa") if metar.press else "N/A",
            "Humidity (%)": calculate_humidity(metar.temp.value("C"), metar.dewpt.value("C")),        })
    except Exception as e:
        print("Error parsing METAR:", e)

# Convert structured METAR data to DataFrame
df = pd.DataFrame(data)
df.to_csv("December_Metar.csv", index=False)
# Display first few rows to verify
print(df.head())



Error parsing METAR: Unparsed groups in body 'TAF 0106/0206 PROB30 0108/0116 -SHRA BKN012 0109/0111 23012KT 0121/0123 28012KT 0122/0206 -SHRA BKN012 0200/0206 31015G25KT' while processing 'TAF EIDW 010500Z 0106/0206 20005KT 9999 FEW005 SCT018
         PROB30 TEMPO 0108/0116 -SHRA BKN012
         BECMG 0109/0111 23012KT
         BECMG 0121/0123 28012KT
         TEMPO 0122/0206 -SHRA BKN012
         TEMPO 0200/0206 31015G25KT='
Error parsing METAR: Unparsed groups in body 'TAF 0112/0212 PROB30 0112/0115 -SHRA BKN012 0115/0118 -SHRA BKN012 FEW020CB 0116/0118 26012KT 0120/0201 27015G26KT -SHRA BKN012 0200/0202 30014KT 0201/0205 30015G26KT PROB30 0201/0204 -SHRA BKN012 PROB30 0209/0212 31014G25KT' while processing 'TAF EIDW 011100Z 0112/0212 23012KT 9999 FEW012 BKN025
         PROB30 TEMPO 0112/0115 -SHRA BKN012
         TEMPO 0115/0118 -SHRA BKN012 FEW020CB
         BECMG 0116/0118 26012KT
         TEMPO 0120/0201 27015G26KT -SHRA BKN012
         BECMG 0200/0202 30014KT
         TEMPO 0201

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from metar import Metar  # Ensure you have the metar package installed

# Function to calculate relative humidity
def calculate_humidity(temp, dew_point):
    if temp == "N/A" or dew_point == "N/A":
        return "N/A"
    return round(100 * (np.exp((17.625 * dew_point) / (243.04 + dew_point)) /
                        np.exp((17.625 * temp) / (243.04 + temp))), 2)

# List of URLs to scrape METAR data from
urls = ["https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=SA&ord=DIR&nil=SI&fmt=html&ano=2024&mes=01&day=01&hora=00&anof=2024&mesf=01&dayf=31&horaf=23&minf=59&send=send",
"https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=SA&ord=DIR&nil=SI&fmt=html&ano=2024&mes=02&day=01&hora=00&anof=2024&mesf=02&dayf=29&horaf=23&minf=59&send=send",
"https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=2024&mes=03&day=01&hora=00&anof=2024&mesf=03&dayf=31&horaf=23&minf=59&send=send",
"https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=ALL&ord=DIR&nil=SI&fmt=html&ano=2024&mes=04&day=01&hora=00&anof=2024&mesf=04&dayf=30&horaf=23&minf=59&send=send",
"https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=ALL&ord=DIR&nil=SI&fmt=html&ano=2024&mes=05&day=01&hora=00&anof=2024&mesf=05&dayf=31&horaf=23&minf=59&send=send",
"https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=ALL&ord=DIR&nil=SI&fmt=html&ano=2024&mes=06&day=01&hora=00&anof=2024&mesf=06&dayf=30&horaf=23&minf=59&send=send",
"https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=ALL&ord=DIR&nil=SI&fmt=html&ano=2024&mes=07&day=01&hora=00&anof=2024&mesf=07&dayf=31&horaf=23&minf=59&send=send",
"https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=ALL&ord=DIR&nil=SI&fmt=html&ano=2024&mes=08&day=01&hora=00&anof=2024&mesf=08&dayf=31&horaf=23&minf=59&send=send",
"https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=ALL&ord=DIR&nil=SI&fmt=html&ano=2024&mes=09&day=01&hora=00&anof=2024&mesf=09&dayf=30&horaf=23&minf=59&send=send",
"https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=ALL&ord=DIR&nil=SI&fmt=html&ano=2024&mes=10&day=01&hora=00&anof=2024&mesf=10&dayf=31&horaf=23&minf=59&send=send",
"https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=ALL&ord=DIR&nil=SI&fmt=html&ano=2024&mes=11&day=01&hora=00&anof=2024&mesf=11&dayf=30&horaf=23&minf=59&send=send",
"https://www.ogimet.com/display_metars2.php?lang=en&lugar=EIDW&tipo=ALL&ord=DIR&nil=SI&fmt=html&ano=2024&mes=12&day=01&hora=00&anof=2024&mesf=12&dayf=31&horaf=23&minf=59&send=send"
]

data = []  # Store parsed METAR data

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract METAR strings from <pre> tags
    metar_strings = [pre.get_text().strip() for pre in soup.find_all('pre')]

    for metar_string in metar_strings:
        try:
            metar = Metar.Metar(metar_string)  # Parse METAR data
            temp_c = metar.temp.value("C") if metar.temp else "N/A"
            dew_c = metar.dewpt.value("C") if metar.dewpt else "N/A"

            data.append({
                "Station": metar.station_id,
                "Observation Time": metar.time.strftime('%Y-%m-%d %H:%M:%S') if metar.time else "N/A",
                "Wind Speed (KT)": metar.wind_speed.value("KT") if metar.wind_speed else "N/A",
                "Visibility (m)": metar.vis.value("m") if metar.vis else "N/A",
                "Temperature (C)": temp_c,
                "Dew Point (C)": dew_c,
                "Humidity (%)": calculate_humidity(temp_c, dew_c),
                "Pressure (hPa)": metar.press.value("hPa") if metar.press else "N/A",
            })
        except Exception as e:
            print("Error parsing METAR:", e)

# Convert to DataFrame and append to CSV
df = pd.DataFrame(data)
df.to_csv("metar_data.csv", mode='a', index=False, header=not pd.io.common.file_exists("metar_data.csv"))

print("CSV file updated successfully with new METAR data!")


Error parsing METAR: Unparsed groups in body '800 FG' while processing 'METAR EIDW 050030Z 27009KT 1200 BR FEW001 BKN002 04/04 Q1000 TEMPO 800 FG='
Error parsing METAR: Unparsed groups in body '800 FG BKN001' while processing 'METAR EIDW 141000Z 29010KT 2000 BR SCT001 BKN002 BKN027 01/01 Q1017 TEMPO 800 FG BKN001='
Error parsing METAR: Unparsed groups in body '170V240' while processing 'METAR EIDW 270630Z 20010KT 9999 170V240 FEW016 BKN028 07/03 Q1026 NOSIG='
Error parsing METAR: Unparsed groups in body 'FDW013' while processing 'METAR EIDW 061830Z 32006KT 9999 FDW013 SCT050 BKN140 02/00 Q1006 NOSIG='
Error parsing METAR: Unparsed groups in body '800 FG BKN002' while processing 'METAR EIDW 210230Z 14010KT 110V170 1200 BR FEW002 BKN004 08/07 Q1006 TEMPO 800 FG BKN002='
Error parsing METAR: Unparsed groups in body 'R10R0700U' while processing 'METAR EIDW 111900Z 30003KT 9999 R28L/2000 R10R0700U R28R/2000 MIFG FEW008 03/03 Q1040 NOSIG='
Error parsing METAR: Unparsed groups in body '28L/17

JOINING THE 12 DATASETS TOGETHER AND ORDERING THEM BY DATE AND TIME

In [6]:
import pandas as pd
import os

# Folder containing CSV files
folder_path = "/content/drive/MyDrive/safety"  # Update with the actual path

# Get all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Read and concatenate all CSVs
df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]
merged_df = pd.concat(df_list, ignore_index=True)

# Save the merged file
merged_df.to_csv("/content/drive/MyDrive/safety/merged_output.csv", index=False)


print("All files have been merged successfully!")


All files have been merged successfully!


In [11]:
import pandas as pd

# Load the merged CSV file
df = pd.read_csv("merged_output.csv")

# Convert the 'Observation Time' column to datetime format
df['Observation Time'] = pd.to_datetime(df['Observation Time'], dayfirst=True, errors='coerce')

# Sort by date and time
df = df.sort_values(by='Observation Time')

# Save the sorted file
df.to_csv("/content/drive/MyDrive/safety/sorted_output.csv", index=False)

print("File sorted successfully!")


File sorted successfully!


# Rest API interaction for flight data collection


Data collected from https://aviationstack.com/

In [ ]:
pip install requests

In [ ]:
import requests
import json
import csv

# API details
base_url = "https://api.aviationstack.com/v1/"
endpoint = "flights"
access_key = "90356201bafcece5d0a8f8e7197a7e5e"  # Replace with your actual access key

# Query parameters to filter flights departing from Dublin Airport (DUB)
params = {
    'access_key': access_key,
    'dep_iata': 'DUB',  # Departures from Dublin
    'limit': 5  # Number of records to fetch
}

# Send request to the API
response = requests.get(base_url + endpoint, params=params)

# Check response
if response.status_code == 200:
    data = response.json()

    # Define CSV file name
    csv_filename = "flights_data.csv"

    # Define CSV headers
    headers = [
        "Flight Number", "Airline", "Aircraft Model", "Registration",
        "Departure Airport", "Scheduled Departure", "Estimated Departure",
        "Actual Departure", "Departure Terminal", "Departure Gate", "Departure Delay",
        "Arrival Airport", "Scheduled Arrival", "Estimated Arrival",
        "Actual Arrival", "Arrival Terminal", "Arrival Gate", "Arrival Delay",
        "Flight Status"
    ]

    # Open CSV file and write data
    with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(headers)  # Write header row

        for flight in data.get('data', []):
            def safe_get(dictionary, key, default="N/A"):
                return dictionary.get(key, default) if dictionary else default

            row = [
                safe_get(flight.get('flight'), 'iata'),
                safe_get(flight.get('airline'), 'name'),
                safe_get(flight.get('aircraft'), 'model'),
                safe_get(flight.get('aircraft'), 'registration'),

                safe_get(flight.get('departure'), 'airport'),
                safe_get(flight.get('departure'), 'scheduled'),
                safe_get(flight.get('departure'), 'estimated'),
                safe_get(flight.get('departure'), 'actual'),
                safe_get(flight.get('departure'), 'terminal'),
                safe_get(flight.get('departure'), 'gate'),
                safe_get(flight.get('departure'), 'delay'),

                safe_get(flight.get('arrival'), 'airport'),
                safe_get(flight.get('arrival'), 'scheduled'),
                safe_get(flight.get('arrival'), 'estimated'),
                safe_get(flight.get('arrival'), 'actual'),
                safe_get(flight.get('arrival'), 'terminal'),
                safe_get(flight.get('arrival'), 'gate'),
                safe_get(flight.get('arrival'), 'delay'),

                safe_get(flight, 'flight_status')
            ]

            writer.writerow(row)  # Write flight data row

    print(f"Data successfully saved to {csv_filename}")

else:
    print(f"Error: {response.status_code}, {response.text}")